In [24]:
# Import Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline

In [25]:
# Set variables for data directory, results directory, and file name.  Import CSV file, trim off top row, drop unneccessary columns, rename columns.
datadirname = '/Users/nhein/Desktop/Coding/Project/Data/'
resultsdirname = '/Users/nhein/Desktop/Coding/Project/Results/'
datafile = 'Ch5-36C.csv'

df = pd.read_csv(datadirname + datafile, skiprows=1)
df.drop(df.columns[[0, 4, 5, 6]], axis=1, inplace= True)
df.columns = ['Full_Date', 'Temp_C', 'RH']

# Change dates to datetime and create new columns with date data needed for calculations
df.Full_Date = pd.to_datetime(df.Full_Date)
df.insert(1,'Date', df.Full_Date.dt.date)
df.insert(2,'Time', df.Full_Date.dt.time)
df.insert(3, 'Hour', df.Full_Date.dt.hour)

# drop rows witn NaNs
df = df.dropna()

In [26]:
# Defining VPD Function
def vpdfun (T_Grid, RH_Grid): 
    '''
    Uses the inputs of Temperature (Celsius) and relative humidity (%RH) and outputs the VPD (kPa).
    Inputs must be in numpy 2D array T in Celsius and RH in %RH with same dimensions.
    Author: Nathan Hein
    Date: 03/03/2019    
    '''
    esat = .611 * (np.exp((T_Grid * 17.5) / (T_Grid + 241))) # Sat. Vapor Pressure
    eact = esat * (RH_Grid / 100)                            # Actual Vapor Pressure
    vpd = esat - eact                                        # Vapr Pressure Deficit
    vpd = np.round(vpd, 2) 
    return vpd

# Defining Temp and RH variables
T_Grid = df.Temp_C
RH_Grid = df.RH

# Calling function and creating new column in dataframe with VPD.
VPD = vpdfun(T_Grid, RH_Grid)
df.insert(6,'VPD', VPD)

In [27]:
# Defining function that averages the VPD values over a given hour of the day for the entire data set.
def monthlyvpd ():
    '''
    Takes the calculated VPD and averages all values that were measured during each hour of the day throughout the data period.
    Records the results in a new dataframe.
    Inputs: VPD as an integer from a dataframe created from a csv file and results are the same units as inputs (normally kPA).  
    Author: Nathan Hein
    Date: 03/11/19
    '''
    cols = ['Hour', 'VPD']
    rows = []
    xdf = pd.DataFrame(rows, columns=cols)
    vpdlist = []
    for x in range(0,24):
        vpd = df[df['Hour']==x]['VPD']
        vpdlist.append(vpd)
        avg = np.mean(vpdlist)
        xdf = xdf.append({'Hour': x, 'VPD': avg}, ignore_index=True)
        vpdlist = []
    return(xdf)

MonthlyHourlyVPD = monthlyvpd()

In [28]:
# Defining Function that average the temp values over a given hour of the day for the entire data set.
def monthlytemp ():
    '''
    Calculates the average temperature for each hour of the day over the entire data set.
    Inputs: Temperature as an integer in a dataframe created from a csv file.
    Outputs: Temperature as an integer and a new dataframe with average temeperature values and the hour.
    Author: Nathan Hein
    Date: 03/11/19
    '''
    cols = ['Hour', 'Temp']
    rows = []
    ydf = pd.DataFrame(rows, columns=cols)
    templist = []
    for x in range(0,24):
        temp = df[df['Hour']==x]['Temp_C']
        templist.append(temp)
        avg = np.mean(templist)
        ydf = ydf.append({'Hour': x, 'Temp': avg}, ignore_index=True)
        templist = []
    return(ydf)

MonthlyHourlyTemp = monthlytemp()

In [29]:
# Defining function that averages the RH values over a given hour of the day for the entire data set.
def monthlyrh ():
    '''
    Calculates the average RH value for each hour of the day over the entire data set.
    Input: RH from a dataframe created from the imported CSV file
    Output: Hourly average RH in a new dataframe and the hour.
    Author: Nathan Hein
    Date: 03/11/19
    '''
    
    cols = ['Hour', 'RH']
    rows = []
    zdf = pd.DataFrame(rows, columns=cols)
    rhlist = []
    for x in range(0,24):
        rh = df[df['Hour']==x]['RH']
        rhlist.append(rh)
        avg = np.mean(rhlist)
        zdf = zdf.append({'Hour': x, 'RH': avg}, ignore_index=True)
        rhlist = []
    return(zdf)

MonthlyHourlyRH = monthlyrh()

In [30]:
# Creates a list of the different Data types data frames and merges all the resuls data frames into one.
data_frames = [MonthlyHourlyTemp, MonthlyHourlyRH, MonthlyHourlyVPD]
MHA_Merged = reduce(lambda left,right: pd.merge(left,right,on=['Hour'],how='outer'), data_frames)
MHA_Merged

,Hour,Temp,RH,VPD
0,0.0,15.303431,71.103138,0.496383
1,1.0,15.283346,71.214032,0.494362
2,2.0,15.273516,71.150495,0.495319
3,3.0,15.262032,71.169117,0.494255
4,4.0,15.256809,71.142122,0.494840
5,5.0,15.523702,70.248399,0.518085
6,6.0,20.637186,69.975819,0.725426
7,7.0,26.904883,67.894213,1.094787
8,8.0,31.241665,66.876239,1.430213
9,9.0,33.378484,67.242644,1.582128


In [31]:
df

,Full_Date,Date,Time,Hour,Temp_C,RH,VPD
0,2018-11-17 12:37:17,2018-11-17,12:37:17,12,21.680,29.689,1.82
1,2018-11-17 12:52:17,2018-11-17,12:52:17,12,34.527,96.867,0.17
2,2018-11-17 13:07:17,2018-11-17,13:07:17,13,35.404,84.067,0.92
3,2018-11-17 13:22:17,2018-11-17,13:22:17,13,35.671,79.317,1.21
4,2018-11-17 13:37:17,2018-11-17,13:37:17,13,35.859,74.078,1.53
5,2018-11-17 13:52:17,2018-11-17,13:52:17,13,35.832,71.706,1.67
6,2018-11-17 14:07:17,2018-11-17,14:07:17,14,35.859,70.826,1.72
7,2018-11-17 14:22:17,2018-11-17,14:22:17,14,35.886,70.477,1.74
8,2018-11-17 14:37:17,2018-11-17,14:37:17,14,35.967,70.129,1.77
9,2018-11-17 14:52:17,2018-11-17,14:52:17,14,35.994,70.463,1.75
